# 1. Intro to NAO

Here, we will see how to connect to NAO and make him speak.

First, we need to connect to NAO. Execute the code below.

In [ ]:
%pip install ipywidgets==8.0.7  
%pip install ipynao==0.8.9
%pip install naowidgets==0.1.1
import asyncio
import ipynao
from naowidgets import *
nao = ipynao.Robot()
nao.connect("nao.local") # Set your IP address here
nao

Now, let's test the connection by making NAO speak:

In [ ]:
show(nao.ALTextToSpeech.say("I am indeed connected"))

## 1.a - Making sure NAO is in the right state.

We need to make sure NAO's "Autonomous Life" is off, but that his motors are on. Execute these two cells:

In [ ]:
show(nao.ALAutonomousLife.setState("disabled"))

In [ ]:
show(nao.ALMotion.wakeUp())

## 1.b - Animated Speech

Contrast these two calls:

In [ ]:
show(nao.ALTextToSpeech.say("This is with text to speech"))

In [ ]:
show(nao.ALAnimatedSpeech.say("And this is with animated speech"))

In [ ]:
show(nao.ALAnimatedSpeech.say("I refuse to do that. No way! I do not want to."))

As you can see, with AnimatedSpeech, NAO will move his arms as he talks.

## 1.c - Tweaking NAO's voice

You can add **voice tags** - see [the documentation](http://doc.aldebaran.com/2-8/naoqi/audio/altexttospeech-tuto.html#using-tags-for-voice-tuning) for a full reference. Here are some examples:

In [ ]:
# Say the sentence with a pitch of +50%​
show(nao.ALAnimatedSpeech.say("\\vct=150\\Hello my friends"))

In [ ]:
# Say the sentence 50% slower than normal speed​
show(nao.ALAnimatedSpeech.say("\\rspd=50\\hello my friends"))

In [ ]:
# Say the sentence with a volume of 50%
show(nao.ALAnimatedSpeech.say("\\vol=50\\Hello my friends"))

In [ ]:
# Change the tone (available: normal, joyful, didactic)
show(nao.ALAnimatedSpeech.say("\\style=joyful\\ Today I am feeling happy."))

In [ ]:
# Reset with \\rst\\
show(nao.ALAnimatedSpeech.say("\\vct=150\\\\rspd=50\\Hello my friends.\\rst\\ How are you ?")

In [ ]:
show(nao.ALAnimatedSpeech.say("(Your sentence here)"))

## 1.d - Sequencing actions

Now that we have learned how a few basic commands to make NAO speak, let's see a fgew more and how to combine them.

Technically, each call to a NAOqi function like ALTextToSpeech.say returns a "coroutine".

So far, we have used "show", which simply shows the result of one coroutine. If we want to sequence several, we can use "as_widget" like this:

In [ ]:
@as_widget
async def try_leds(print):
    await nao.ALAnimatedSpeech.say("Look at my eyes")
    await nao.ALLeds.rasta(2.0)
    await nao.ALAnimatedSpeech.say("Not bad, right?")
    print("Done.")

Here's an example with even more commands. Now he we disable background movement to make sure NAO holds the pose as he speaks:

In [ ]:
@as_widget
async def little_speech(print):
    await nao.ALAnimatedSpeech.say("So...")
    await nao.ALBackgroundMovement.setEnabled(False)
    scratch_head = [-1.5187020301818848, 0.440216064453125, -0.14270401000976562, -1.3222661018371582, -1.3821759223937988, 0.00839996337890625]
    await nao.ALMotion.angleInterpolationWithSpeed("LArm", scratch_head, 0.5)
    await nao.ALTextToSpeech.say("What's going on here?")
    await nao.ALBackgroundMovement.setEnabled(True)
    print("Done.")

This can also be useful to get return values from functions:

In [ ]:
@as_widget
async def print_joints(print):
    arm_joints = await nao.ALMotion.getJointNames("LArm")
    print("Arm joints:", arm_joints)

Here's an example with a loop:

In [ ]:
@as_widget
async def list_languages(print):
    languages = await nao.ALTextToSpeech.getAvailableLanguages()
    print("Languages:", languages)
    await nao.ALAnimatedSpeech.say(f"I can speak {len(languages)} languages")
    for language in languages:
        await nao.ALAnimatedSpeech.say(language)

# 2 - Dialogue

QiChat is a language for defining dialogues - what the robot can hear, what he can answer.

Using AI Chat requires Autonomous Life to be activated:

In [ ]:
show(nao.ALAutonomousLife.setState("solitary"))

## 2.a A simple dialogue

Create the following dialogue:

In [ ]:
show(nao.ALAIChat.set_topic("""
u:(hello) hello, sorry for not hearing you earlier it was my fault ?

u:(what are we doing today?) We are programming a robot. I wonder who that is?

u:(who are you?) I am NAO the little robot!

u:(how are you) not that bad!
"""))

## 2.b Testing the dialogue

To switch to dialogue mode, press NAO's right bumper (that is, the one on your right). NOTE: this is a not a standard feature of NAO, but one added in this package to make testing this dialogue more convenient.

You can press the other bumper (to your right) to quit the app and return to Autonomous Life; note that this might trigger the robot's default dialogue if you have any installed, so be sure to keep track of the robot's state (recognizable by the blue feet).

## 2.c More advanced dialogue

Now that the basic dialogue works, you can try more advanced things - see [the documentation](http://doc.aldebaran.com/2-8/naoqi/interaction/dialog/dialog-syntax_full.html) for a full reference.

In [ ]:
nao.ALAIChat.set_topic("""
u:(I {really} like _[tea coffee]) Good to know $drink=$1

u:(My name is _*) Pleased to meet you $1 $name=$1

u:(Do you know me)
 ^first[
   "Yes, you are $name and you like $drink"
   "Yes, you are $name"
   "I know you like $drink"
   "No, I don't know you"
 ]
""")

# 3 - AI Chat

This is more advanced chat, powered by OpenAI's services. It can be combined with qichat-based chat, and works the same way.

You can set a prompt with `ALAIChat.set_prompt`:

In [ ]:
show(nao.ALAIChat.set_prompt("""
You are NAO, a friendly humanoid robot.

You are powered by ChatGPT but actually running on a physical NAO robot.
NAO is used in education at all levels (primary to higher ed and research).
NAO can be programmed with Choregraphe, Python, C++ and Jupyter Notebook.
NAO is also sometimes used in retirement homes and special education.

Be sure to speak in short simple sentences, like a child.
"""))

In [ ]:
show(nao.ALAIChat.set_prompt("""
You are NAO, a friendly humanoid robot.

You are here to explain german idealism."""))

In [ ]:
show(nao.ALAIChat.set_prompt("""
You are NAO, a friendly humanoid robot.

You are powered by ChatGPT but actually running on a physical NAO robot.
You are used in education at all levels (primary to higher ed and research).

You are in an elderly care institution, and can suggest exercise or ask people about their day.

Today is tuesday, the families will visit this afternoon, and there is beans at the cafeteria.

Be sure to speak in short simple sentences, like a child.
"""))

Once this is done, you can switch between qichat mode and "AI mode" by touching NAO's feet's bumpers: 
* The bumper to your left to enter AI mode (the feet should turn green)
* The bumper to your right to return to qichat mode (the feet should be blue)

When in AI mode, NAO will answer all your questions taking that prompt into account.

## 3.b - Troubleshooting

Sometimes the robot will not answer. Some things to check

1) Are the feet green ? If not, you might not be in the test app, try making sure Autonomous Life is activated and press the left foot bumper (blue feet), then again (green feet)

2) Are the eyes blue ? If not, try restarting the test app (note that if the eyes are blue and not the feet, he's not using your dialogue, just a systme one)

3) Is Free speech active ? You can try with this dialogue:

In [ ]:
show(nao.ALAIChat.set_topic("""
u:(repeat _*) you said $1

u:(are you listening) I think so, yes
"""))

If it doesn't work, it could be a network connection problem or, more likely, that Free Speech is not activated on your robot. If it used to work but doesn't anymore, rebooting NAO could help.

In [ ]:
show(nao.ALAIChat.answer("Hey NAO, what do you think of that movie concept?"))

# 4 - Motion

Now let's make NAO move! This can be done in several ways: joint control and motion control.

But first, we should disable life and wake up (i.e. turn all motors on):

In [ ]:
@as_widget
async def prepare_for_motion(print):
    await nao.ALAutonomousLife.setState("disabled")
    await nao.ALMotion.wakeUp()
    print("NAO should be ready for motion control!")
    

## 4.a - Joint control

The `ALMotion` service allows to control individual joints: their stiffness, and their angles.

For example, let's make NAO take a "scratch his head" pose:

In [ ]:
scratch_head = [-1.5187020301818848, 0.440216064453125, -0.14270401000976562, -1.3222661018371582, -1.3821759223937988, 0.00839996337890625]
show(nao.ALMotion.setAngles("LArm", scratch_head, 0.5))

In [ ]:
hand_on_hip = [1.4495880603790283, 0.817579984664917, 0.07512402534484863, -1.5585020780563354, -1.8362398147583008, 0.011199951171875]
show(nao.ALMotion.setAngles("LArm", hand_on_hip, 0.5))

Instead of setAngles, one can use `angleInterpolationWithSpeed`, which gives a slightly nice movement:

In [ ]:
show(nao.ALMotion.angleInterpolationWithSpeed("LArm", scratch_head, 0.5))

## 4.b - Dedicated control widgets

Using a dedicated widget can make arm control easier. With this Arm Controller widget, you can set a series of poses:

In [ ]:
larm = LeftArmController(nao)

A more advanced widget can control the whole torso:

In [ ]:
torso = NaoTorsoController(nao)

## 4.c - Walking around

You can make NAO walk around with ALMotion.moveTo (x, y, theta) where x is forward.

WARNING: do not do this if NAO is on a table!

In [ ]:
show(nao.ALMotion.moveTo(0.5, 0., 0.0))

# 5 - AI-driven actions

Now, let's get back to ChatGPT. So far, we have only used it in a straightforward way, to make NAO speak. But actually, it can also send back **commands** that NAO can execute, to make him move, walk, etc.

## 5.a - Discovering action tools

These commands correspond to **tools** and tools can be activated on `ALAIChat`. You can see which ones are available with `ALAIChat.get_available_tools()`.

In [ ]:
show(nao.ALAIChat.get_available_tools())

You can activate / deactivate tools with `ALAIChat.activate_tool` and `ALAIChat.deactivate_tool`. For example, let's activate "rasta", which is a useful one for testing.

In [ ]:
show(nao.ALAIChat.activate_tool("rasta"))

You can then try this by switching to AI mode ... but you can also try directly sending a command to the robot, useful for testing:

In [ ]:
show(nao.ALAIChat.answer("Make your LEDs move for a few seconds in a rasta pattern."))

Note that a useful tool to activate is "say", as that one allows an answer that treats speech as an action that can be mixed among the others:

In [ ]:
show(nao.ALAIChat.activate_tool("change_posture"))

## 5.b - Combining several tools

Tool calls can be combined; let's activate much more:

In [ ]:
show(nao.ALAIChat.set_prompt("Your are a simple, obedient robot. You can perform actions with tool calls (say, rasta, think), including several in a row. When speeking, stay very short."))

In [ ]:
#TOOL_TO_ACTIVATE = ["rasta", "say", "think", "play_animation", "change_posture", "custom_pose"]
TOOL_TO_ACTIVATE = ["rasta", "say", "think"]

@as_widget
async def activate_tools(print):
    for tool_name in TOOL_TO_ACTIVATE:
        success = await nao.ALAIChat.activate_tool(tool_name)
        print("Activate", tool_name, "status:", success)
    print("Done with activating tools")

You can try these out directly with `ALAIChat.answer`:

In [ ]:
show(nao.ALAIChat.answer("Count to three, for each one alternating between saying that number and doing a rasta eye pattern for that many seconds."))

You can use this function to inspect what the robot did:

In [ ]:
show(nao.ALAIChat.get_printable_last_actions())

In [ ]:
show(nao.ALAIChat.get_active_tools())

You can experiment a bit with the other tools, for example, use `move_to` to make a NAO that can walk around.

You can deactivate tools with this:

In [ ]:
@as_widget
async def deactivate_tools(print):
    for tool_name in await nao.ALAIChat.get_active_tools():
        success = await nao.ALAIChat.deactivate_tool(tool_name)
        print("Deactivate", tool_name, "status:", success)
    print("Done with deactivating tools")

## 5.c - Simple custom poses

`custom_pose` is a special tool that allows you to nbame and define your own pose, that can then be triggered by the LLM.

You can do that with `ALAIChat.add_custom_pose` that takes as parameter:
 - the name you want to give the pose - this one is important, because that's how
 - the list of joints
 - a list of poses, each of which should be she same size as the list of joints.

For example:

In [ ]:
show(nao.ALAIChat.add_custom_pose("shake head", ["HeadYaw"], [[-1.0], [1.0], [.0]]))

If you haven't already, be sure to activate the custom pose tool:

In [ ]:
show(nao.ALAIChat.activate_tool("custom_pose"))

You can now ask NAO to shake his head, either directly or with this command:

In [ ]:
show(nao.ALAIChat.answer("Say something sad, then shake your head."))

## 5.d - More custom poses

Of course, manually defining each single joint can be quite some cumbersone; so instead let's use the torso control widget we used earlier.

But first, let's make a helper function to disable/enable autonomous abilities:

In [ ]:
enabled = True
@as_widget
async def set_abilities(print):
    print("Setting abilities enabled=", enabled)
    await nao.ALBackgroundMovement.setEnabled(enabled)
    await nao.ALListeningMovement.setEnabled(enabled)
    await nao.ALBasicAwareness.setEnabled(enabled)
    print("Setting abilities done")


Now let's create our widget:

In [ ]:
torso = NaoTorsoController(nao)

You can the this widget to define a pose, and save it here:

In [ ]:
show(nao.ALAIChat.add_custom_pose("show off", torso.all_joints, torso.poses))

You can now create any number of poses, and the LLM will be capable of using them.

# 6 - Non-speech inputs

NAO's default action loop is listening for speech, then answering accordingly.

For this section, we will stop using dialogue, and use NAO's sensors instead.

To do that, let's stop AutonomousLife, so that NAO will not listen any more:

In [ ]:
@as_widget
async def stopLife(print):
    await nao.ALAutonomousLife.setState("disabled")
    await nao.ALMotion.wakeUp()

## 6.a Manually triggering with a system message

Here instead of `answer` we will use `answer_sysem_message`, which is similar but used to describe what the robot perceived, still as English. But NAO will not treat answer it as if it was dialogue.

In [ ]:
show(nao.ALAIChat.answer_system_message("You are now alone. Time elapsed=25s."))

## 6.b Your own loop.

You can then use this to create your own interaction loop, for example this one, that will make NAO walk around.

Make sure to put him on the ground first!

In [ ]:
@as_widget
async def sonarLoop(print):
    for i in range(5):        
        left = await nao.ALMemory.getData("Device/SubDeviceList/US/Left/Sensor/Value")
        right = await nao.ALMemory.getData("Device/SubDeviceList/US/Right/Sensor/Value")
        context_desc = f"Left sonar: {left:.1f}m, Right sonar: {right:.1f}m"
        print("Context:", context_desc)
        r = await nao.ALAIChat.answer_system_message(context_desc)
        print(f"Step {i} success: {r}")
        actions = await nao.ALAIChat.get_printable_last_actions()
        print(actions)
        await nao.ALRobotPosture.goToPosture("Stand", 0.5) # Make sure the robot ends in a good posture
    await nao.ALTextToSpeech.say("loop done")

## 6.c Other sensors

In this loop, we use sensors, but you could actually collect a lot of information, such as:

 * NAO's (estimated) position in the world [ALMotion.getPosition](http://doc.aldebaran.com/2-8/naoqi/motion/control-cartesian-api.html#ALMotionProxy::getPosition__ssCR.iCR.bCR)
 * Detected people: with [PeoplePerception](http://doc.aldebaran.com/2-8/naoqi/peopleperception/alpeopleperception-api.html#events-lists)
 * Detected people's face expressions with [ALGazeAnalysis](http://doc.aldebaran.com/2-8/naoqi/peopleperception/algazeanalysis.html#algazeanalysis) and [ALFaceCharacteristics](http://doc.aldebaran.com/2-8/naoqi/peopleperception/alfacecharacteristics.html#alfacecharacteristics)
 * NAO's standing / sitting pose with [ALRobotPosture](http://doc.aldebaran.com/2-8/naoqi/motion/alrobotposture.html)